<h1>11  Time Series</h1>

In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision=4, suppress=True)

<p>Time series data is an important form of structured data in many different fields, such as finance, economics, ecology, neuroscience, and physics. Anything that is recorded repeatedly at many points in time forms a time series. Many time series are <em>fixed frequency</em>, which is to say that data points occur at regular intervals according to some rule, such as every 15 seconds, every 5 minutes, or once per month. Time series can also be <em>irregular</em> without a fixed unit of time or offset between units. How you mark and refer to time series data depends on the application, and you may have one of the following:</p>

<dl>
    <dt>Timestamps</dt>
    <dd>Specific instants in time.</dd>
    <dt>Fixed periods</dt>
    <dd>Such as the whole month of January 2017, or the whole year 2020.</dd>
    <dt>Intervals of time</dt>
    <dd>Indicated by a start and end timestamp. Periods can be thought of as special cases of intervals.</dd>
    <dt>Experiment or elapsed time</dt>
    <dd>Each timestamp is a measure of time relative to a particular start time (e.g., the diameter of a cookie baking each second since being placed in the oven), starting from 0.</dd>
</dl>

<p>In this chapter, I am mainly concerned with time series in the first three categories, though many of the techniques can be applied to experimental time series where the index may be an integer or floating-point number indicating elapsed time from the start of the experiment. The simplest kind of time series is indexed by timestamp.</p>


<p><strong>Tip</strong><br />
pandas also supports indexes based on timedeltas, which can be a useful way of representing experiment or elapsed time. We do not explore timedelta indexes in this book, but you can learn more in the <a href="https://pandas.pydata.org/">pandas documentation</a>.</p>

<p>pandas provides many built-in time series tools and algorithms. You can efficiently work with large time series, and slice and dice, aggregate, and resample irregular- and fixed-frequency time series. Some of these tools are useful for financial and economics applications, but you could certainly use them to analyze server log data, too.</p>

<p>As with the rest of the chapters, we start by importing NumPy and pandas:</p>

In [2]:
import numpy as np
import pandas as pd

<h2>11.1 Date and Time Data Types and Tools</h2>

<p>The Python standard library includes data types for date and time data, as well as calendar-related functionality. The <code>datetime</code>, <code>time</code>, and <code>calendar</code> modules are the main places to start. The <code>datetime.datetime</code> type, or simply <code>datetime</code>, is widely used:</p>

In [3]:
from datetime import datetime
now = datetime.now()
now

In [4]:
now.year, now.month, now.day

<p><code>datetime</code> stores both the date and time down to the microsecond. <code>datetime.timedelta</code>, or simply <code>timedelta</code>, represents the temporal difference between two <code>datetime</code> objects:</p>

In [5]:
delta = datetime(2011, 1, 7) - datetime(2008, 6, 24, 8, 15)

delta

In [6]:
delta.days

In [7]:
delta.seconds

<p>You can add (or subtract) a <code>timedelta</code> or multiple thereof to a <code>datetime</code> object to yield a new shifted object:</p>

In [8]:
from datetime import timedelta
start = datetime(2011, 1, 7)
start + timedelta(12)

In [9]:
start - 2 * timedelta(12)

<p><a href="#table_11.1">Table 11.1</a> summarizes the data types in the <code>datetime</code> module. While this chapter is mainly concerned with the data types in pandas and higher-level time series manipulation, you may encounter the <code>datetime</code>-based types in many other places in Python in the wild.</p>

<table id="table_11.1">
    <caption>Table 11.1: Types in the <code>datetime</code> module</caption>
    <tr>
        <th>Type</th>
        <th>Descripction</th>
    </tr>
    <tr>
        <td><code>date</code></td>
        <td>Store calendar date (year, month, day) using the Gregorian calendar</td>
    </tr>
    <tr>
        <td><code>time</code></td>
        <td>Store time of day as hours, minutes, seconds, and microseconds</td>
    </tr>
    <tr>
        <td><code>datetime</code></td>
        <td>Store both date and time</td>
    </tr>
    <tr>
        <td><code>timedelta</code></td>
        <td>The difference between two <code>datetime</code> values (as days, seconds, and microseconds)</td>
    </tr>
    <tr>
        <td><code>tzinfo</code></td>
        <td>Base type for storing time zone information</td>
    </tr>
</table>

<h3>Converting Between String and Datetime</h3>

<p>You can format <code>datetime</code> objects and pandas <code>Timestamp</code> objects, which I’ll introduce later, as strings using <code>str</code> or the <code>strftime</code> method, passing a format specification:</p>

In [10]:
stamp = datetime(2011, 1, 3)
str(stamp)

In [11]:
stamp.strftime("%Y-%m-%d")

<p>See <a href="#table_11.2">Table 11.2</a> for a complete list of the format codes.</p>

<table id="table_11.2">
    <caption>Table 11.2: <code>datetime</code> format specification (ISO C89 compatible)</caption>
    <tr>
        <th>Type</th>
        <th>Description</th>
    </tr>
    <tr>
        <td><code>%Y</code></td>
        <td>Four-digit year</td>
    </tr>
    <tr>
        <td><code>%y</code></td>
        <td>Two-digit year</td>
    </tr>
    <tr>
        <td><code>%m</code></td>
        <td>Two-digit month [01, 12]</td>
    </tr>
    <tr>
        <td><code>%d</code></td>
        <td>Two-digit day [01, 31]</td>
    </tr>
    <tr>
        <td><code>%H</code></td>
        <td>Hour (24-hour clock) [00, 23]</td>
    </tr>
    <tr>
        <td><code>%I</code></td>
        <td>Hour (12-hour clock) [01, 12]</td>
    </tr>
    <tr>
        <td><code>%M</code></td>
        <td>Two-digit minute [00, 59]</td>
    </tr>
    <tr>
        <td><code>%S</code></td>
        <td>Second [00, 61] (seconds 60, 61 account for leap seconds)</td>
    </tr>
    <tr>
        <td><code>%f</code></td>
        <td>Microsecond as an integer, zero-padded (from 000000 to 999999)</td>
    </tr>
    <tr>
        <td><code>%j</code></td>
        <td>Day of the year as a zero-padded integer (from 001 to 336)</td>
    </tr>
    <tr>
        <td><code>%w</code></td>
        <td>Weekday as an integer [0 (Sunday), 6]</td>
    </tr>
    <tr>
        <td><code>%u</code></td>
        <td>Weekday as an integer starting from 1, where 1 is Monday.</td>
    </tr>
    <tr>
        <td><code>%U</code></td>
        <td>Week number of the year [00, 53]; Sunday is considered the first day of the week, and days before the first Sunday of the year are “week 0”</td>
    </tr>
    <tr>
        <td><code>%W</code></td>
        <td>Week number of the year [00, 53]; Monday is considered the first day of the week, and days before the first Monday of the year are “week 0”</td>
    </tr>
    <tr>
        <td><code>%z</code></td>
        <td>UTC time zone offset as <code>+HHMM</code> or <code>-HHMM</code>; empty if time zone naive</td>
    </tr>
    <tr>
        <td><code>%Z</code></td>
        <td>Time zone name as a string, or empty string if no time zone</td>
    </tr>
    <tr>
        <td><code>%F</code></td>
        <td>Shortcut for <code>%Y-%m-%d</code> (e.g., <code>2012-4-18</code>)</td>
    </tr>
    <tr>
        <td><code>%D</code></td>
        <td>Shortcut for <code>%m/%d/%y</code> (e.g., <code>04/18/12</code>)</td>
    </tr>
</table>
<p>You can use many of the same format codes to convert strings to dates using <code>datetime.strptime</code> (but some codes, like <code>%F</code> cannot be used):</p>

In [12]:
value = "2011-01-03"
datetime.strptime(value, "%Y-%m-%d")

In [13]:
datestrs = ["7/6/2011", "8/6/2011"]
[datetime.strptime(x, "%m/%d/%Y") for x in datestrs]

<p><code>datetime.strptime</code> is one way to parse a date with a known format.</p>

<p>pandas is generally oriented toward working with arrays of dates, whether used as an axis index or a column in a DataFrame. The <code>pandas.to_datetime</code> method parses many different kinds of date representations. Standard date formats like ISO 8601 can be parsed quickly:</p>

In [14]:
datestrs = ["2011-07-06 12:00:00", "2011-08-06 00:00:00"]
pd.to_datetime(datestrs)

<p>It also handles values that should be considered missing(<code>None</code>, empty string, etc.):</p>

In [15]:
idx = pd.to_datetime(datestrs + [None])
idx

In [16]:
idx[2]

In [17]:
pd.isna(idx)

<p><code>NaT</code> (Not a Time) is pandas’s null value for timestamp data.</p>

<p><strong>Danger</strong><br />
<code>dateutil.parser</code> is a useful but imperfect tool. Notably, it will recognize some strings as dates that you might prefer that it didn’t; for example, <code>"42"</code> will be parsed as the year <code>2042</code> with today’s calendar date.</p>

<p><code>datetime</code> objects also have a number of locale-specific formatting options for systems in other countries or languages. For example, the abbreviated month names will be different on German or French systems compared with English systems. See <a href="#table_11.3">Table 11.3</a> for a listing.</p>

<table id="table_11.3">
    <caption>Table 11.3: Locale-specific date formatting </caption>
    <tr>
        <th>Type</th>
        <th>Description</th>
    </tr>
    <tr>
        <td><code>%a</code></td>
        <td>Abbreviated weekday name</td>
    </tr>
    <tr>
        <td><code>%A</code></td>
        <td>Full weekday name</td>
    </tr>
    <tr>
        <td><code>%b</code></td>
        <td>Abbreviated month name</td>
    </tr>
    <tr>
        <td><code>%B</code></td>
        <td>Full month name</td>
    </tr>
    <tr>
        <td><code>%c</code></td>
        <td>Full date and time (e.g., ‘Tue 01 May 2012 04:20:57 PM’)</td>
    </tr>
    <tr>
        <td><code>%p</code></td>
        <td>Locale equivalent of AM or PM</td>
    </tr>
    <tr>
        <td><code>%x</code></td>
        <td>Locale-appropriate formatted date (e.g., in the United States, May 1, 2012 yields ‘05/01/2012’)</td>
    </tr>
    <tr>
        <td><code>%X</code></td>
        <td>Locale-appropriate time (e.g., ‘04:24:12 PM’)</td>
    </tr>
</table>

<h2>11.2 Time Series Basics</h2>
<p>A basic kind of time series object in pandas is a Series indexed by timestamps, which is often represented outside of pandas as Python strings or <code>datetime</code> objects:</p>

In [18]:
dates = [datetime(2011, 1, 2), datetime(2011, 1, 5),
         datetime(2011, 1, 7), datetime(2011, 1, 8),
         datetime(2011, 1, 10), datetime(2011, 1, 12)]
ts = pd.Series(np.random.standard_normal(6), index=dates)
ts

<p>Under the hood, these <code>datetime</code> objects have been put in a <code>DatetimeIndex</code>:</p>

In [19]:
ts.index

<p>Like other Series, arithmetic operations between differently indexed time series automatically align on the dates:</p>

In [20]:
ts + ts[::2]

<p>Recall that <code>ts[::2]</code> selects every second element in <code>ts</code>.</p>

<p>pandas stores timestamps using NumPy's <code>datetime64</code> data type at the nanosecond resolution:</p>

In [21]:
ts.index.dtype

<p>Scalar values from a <code>DatetimeIndex</code> are pandas <code>Timestamp</code> objects:</p>

In [22]:
stamp = ts.index[0]
stamp

<p>A <code>pandas.Timestamp</code> can be substituted most places where you would use a <code>datetime</code> object. The reverse is not true, however, because <code>pandas.Timestamp</code> can store nanosecond precision data, while <code>datetime</code> stores only up to microseconds. Additionally, <code>pandas.Timestamp</code> can store frequency information (if any) and understands how to do time zone conversions and other kinds of manipulations. More on both of these things later in <a href="#11.4_time">Time Zone Handling</a>.</p>

<h3>Indexing, Selection, Subsetting</h3>

<p>Time series behaves like any other Series when you are indexing and selecting data based on the label:</p>

In [23]:
stamp = ts.index[2]
ts[stamp]

<p>As a convenience, you can also pass a string that is interpretable as a date:</p>

In [24]:
ts["2011-01-10"]

<p>For longer time series, a year or only a year and month can be passed to easily select slices of data (<code>pandas.date_range</code> is discussed in more detail in <a href="#generating">Generating Date Ranges</a>):</p>

In [25]:
longer_ts = pd.Series(np.random.standard_normal(1000),
                      index=pd.date_range("2000-01-01", periods=1000))
longer_ts

In [26]:
longer_ts["2001"]

<p>Here, the string <code>"2001"</code> is interpreted as a year and selects that time period. This also works if you specify the month:</p>

In [27]:
longer_ts["2001-05"]

<p>Slicing with <code>datetime</code> objects works as well:<p>

In [28]:
ts[datetime(2011, 1, 7):]

In [29]:
ts[datetime(2011, 1, 7):datetime(2011, 1, 10)]

<p>Because most time series data is ordered chronologically, you can slice with timestamps not contained in a time series to perform a range query:</p>

In [30]:
ts

In [31]:
ts["2011-01-06":"2011-01-11"]

<p>As before, you can pass a string date, <code>datetime</code>, or timestamp. Remember that slicing in this manner produces views on the source time series, like slicing NumPy arrays. This means that no data is copied, and modifications on the slice will be reflected in the original data.</p>

<p>There is an equivalent instance method, <code>truncate</code>, that slices a Series between two dates:</p>

In [32]:
ts.truncate(after="2011-01-09")

<p>All of this holds true for DataFrame as well, indexing on its rows:</p>

In [33]:
dates = pd.date_range("2000-01-01", periods=100, freq="W-WED")
long_df = pd.DataFrame(np.random.standard_normal((100, 4)),
                       index=dates,
                       columns=["Colorado", "Texas",
                                "New York", "Ohio"])
long_df.loc["2001-05"]

<h3>Time Series with Duplicate Indices</h3>

<p>In some applications, there may be multiple data observations falling on a particular timestamp. Here is an example:</p>

In [34]:
dates = pd.DatetimeIndex(["2000-01-01", "2000-01-02", "2000-01-02",
                          "2000-01-02", "2000-01-03"])
dup_ts = pd.Series(np.arange(5), index=dates)
dup_ts

<p>We can tell that the index is not unique by checking its <code>is_unique</code> property:</p>

In [35]:
dup_ts.index.is_unique

<p>Indexing into this time series will now either produce scalar values or slices, depending on whether a timestamp is duplicated:</p>

In [36]:
dup_ts["2000-01-03"]  # not duplicated

In [37]:
dup_ts["2000-01-02"]  # duplicated

<p>Suppose you wanted to aggregate the data having non unique timestamps. One way to do this is to use <code>groupby</code> and pass <code>level=0</code> (the one and only level):</p>

In [38]:
grouped = dup_ts.groupby(level=0)
grouped.mean()

In [39]:
grouped.count()

<h2>11.3 Date Ranges, Frequencies, and Shifting</h2>

<p>Generic time series in pandas are assumed to be irregular; that is, they have no fixed frequency. For many applications this is sufficient. However, it’s often desirable to work relative to a fixed frequency, such as daily, monthly, or every 15 minutes, even if that means introducing missing values into a time series. Fortunately, pandas has a full suite of standard time series frequencies and tools for resampling (discussed in more detail later in <a href="#11.6_resampling">Resampling and Frequency Conversion</a>), inferring frequencies, and generating fixed-frequency date ranges. For example, you can convert the sample time series to fixed daily frequency by calling <code>resample</code>:</p>

In [40]:
ts

In [41]:
resampler = ts.resample("D")
resampler

<p>The string <code>"D"</code> is interpreted as daily frequency.</p>

<p>Conversion between frequencies or resampling is a big enough topic to have its own section later (<a href="#11.6_resampling">Resampling and Frequency Conversion</a>). Here, I’ll show you how to use the base frequencies and multiples thereof.</p>

<span id="generating"></span>
<h3>Generating Date Ranges</h3>

<p>While I used it previously without explanation, <code>pandas.date_range</code> is responsible for generating a <code>DatetimeIndex</code> with an indicated length according to a particular frequency:</p>

In [42]:
index = pd.date_range("2012-04-01", "2012-06-01")
index

<p>By default, <code>pandas.date_range</code> generates daily timestamps. If you pass only a start or end date, you must pass a number of periods to generate:</p>

In [43]:
pd.date_range(start="2012-04-01", periods=20)

In [44]:
pd.date_range(end="2012-06-01", periods=20)

<p>The start and end dates define strict boundaries for the generated date index. For example, if you wanted a date index containing the last business day of each month, you would pass the <code>"BM"</code> frequency (business end of month; see a more complete listing of frequencies in <a href="#table_11.4">Table 11.4</a>), and only dates falling on or inside the date interval will be included:</p>

In [45]:
pd.date_range("2000-01-01", "2000-12-01", freq="BM")

<table id="table_11.4">
    <caption>Table 11.4: Base time series frequencies (not comprehensive)</caption>
    <tr>
        <th>Alias</th>
        <th>Offset type</th>
        <th>Description</th>
    </tr>
    <tr>
        <td><code>D</code></td>
        <td><code>Day</code></td>
        <td>Calendar daily</td>
    </tr>
    <tr>
        <td><code>B</code></td>
        <td><code>BusinessDay</code></td>
        <td>Business daily</td>
    </tr>
    <tr>
        <td><code>H</code></td>
        <td><code>Hour</code></td>
        <td>Hourly</td>
    </tr>
    <tr>
        <td><code>T</code> or <code>min</code></td>
        <td><code>Minute</code></td>
        <td>Once a minute</td>
    </tr>
    <tr>
        <td><code>S</code></td>
        <td><code>Second</code></td>
        <td>Once a second</td>
    </tr>
    <tr>
        <td><code>L</code> or <code>ms</code></td>
        <td><code>Milli</code></td>
        <td>Millisecond (1/1,000 of 1 second)</td>
    </tr>
    <tr>
        <td><code>U</code></td>
        <td><code>Micro</code></td>
        <td>Microsecond (1/1,000,000 of 1 second)</td>
    </tr>
    <tr>
        <td><code>M</code></td>
        <td><code>MonthEnd</code></td>
        <td>Last calendar day of month</td>
    </tr>
    <tr>
        <td><code>BM</code></td>
        <td><code>BusinessMonthEnd</code></td>
        <td>Last business day (weekday) of month</td>
    </tr>
    <tr>
        <td><code>MS</code></td>
        <td><code>MonthBegin</code></td>
        <td>First calendar day of month</td>
    </tr>
    <tr>
        <td><code>BMS</code></td>
        <td><code>BusinessMonthBegin</code></td>
        <td>First weekday of month</td>
    </tr>
    <tr>
        <td><code>W-MON, W-TUE, ...</code></td>
        <td><code>Week</code></td>
        <td>Weekly on given day of week (MON, TUE, WED, THU, FRI, SAT, or SUN)</td>
    </tr>
    <tr>
        <td><code>WOM-1MON, WOM-2MON, ...</code></td>
        <td><code>WeekOfMonth</code></td>
        <td>Generate weekly dates in the first, second, third, or fourth week of the month (e.g., <code>WOM-3FRI</code> for the third Friday of each month)</td>
    </tr>
    <tr>
        <td><code>Q-JAN, Q-FEB, ...</code></td>
        <td><code>QuarterEnd</code></td>
        <td>Quarterly dates anchored on last calendar day of each month, for year ending in indicated month (JAN, FEB, MAR, APR, MAY, JUN, JUL, AUG, SEP, OCT, NOV, or DEC)</td>
    </tr>
    <tr>
        <td><code>BQ-JAN, BQ-FEB, ...</code></td>
        <td><code>BusinessQuarterEnd</code></td>
        <td>Quarterly dates anchored on last weekday day of each month, for year ending in indicated month</td>
    </tr>
    <tr>
        <td><code>QS-JAN, QS-FEB, ...</code></td>
        <td><code>QuarterBegin</code></td>
        <td>Quarterly dates anchored on first calendar day of each month, for year ending in indicated month</td>
    </tr>
    <tr>
        <td><code>BQS-JAN, BQS-FEB, ...</code></td>
        <td><code>BusinessQuarterBegin</code></td>
        <td>Quarterly dates anchored on first weekday day of each month, for year ending in indicated month</td>
    </tr>
    <tr>
        <td><code>A-JAN, A-FEB, ...</code></td>
        <td><code>YearEnd</code></td>
        <td>Annual dates anchored on last calendar day of given month (JAN, FEB, MAR, APR, MAY, JUN, JUL, AUG, SEP, OCT, NOV, or DEC)</td>
    </tr>
    <tr>
        <td><code>BA-JAN, BA-FEB, ...</code></td>
        <td><code>BusinessYearEnd</code></td>
        <td>Annual dates anchored on last weekday of given month</td>
    </tr>
    <tr>
        <td><code>AS-JAN, AS-FEB, ...</code></td>
        <td><code>YearBegin</code></td>
        <td>Annual dates anchored on first day of given month</td>
    </tr>
    <tr>
        <td><code>BAS-JAN, BAS-FEB, ...</code></td>
        <td><code>BusinessYearBegin</code></td>
        <td>Annual dates anchored on first weekday of given month</td>
    </tr>
</table>

<p><code>pandas.date_range</code> by default preserves the time (if any) of the start or end timestamp:</p>

In [46]:
pd.date_range("2012-05-02 12:56:31", periods=5)

<p>Sometimes you will have start or end dates with time information but want to generate a set of timestamps <em>normalized</em> to midnight as a convention. To do this, there is a <code>normalize</code> option:</p>

In [47]:
pd.date_range("2012-05-02 12:56:31", periods=5, normalize=True)

<h3>Frequencies and Date Offsets</h3>

<p>Frequencies in pandas are composed of a <em>base frequency</em> and a multiplier. Base frequencies are typically referred to by a string alias, like <code>"M"</code> for monthly or <code>"H"</code> for hourly. For each base frequency, there is an object referred to as a <code>date offset</code>. For example, hourly frequency can be represented with the <code>Hour</code> class:</p>

In [48]:
from pandas.tseries.offsets import Hour, Minute
hour = Hour()
hour

<p>You can define a multiple of an offset by passing an integer:</p>

In [49]:
four_hours = Hour(4)
four_hours

<p>In most applications, you would never need to explicitly create one of these objects; instead you'd use a string alias like <code>"H"</code> or <code>"4H"</code>. Putting an integer before the base frequency creates a multiple:</p>

In [50]:
pd.date_range("2000-01-01", "2000-01-03 23:59", freq="4H")

<p>Many offsets can be combined by addition</p>

In [51]:
Hour(2) + Minute(30)

<p>Similarly, you can pass frequency strings, like <code>"1h30min"</code>, that will effectively be parsed to the same expression:</p>

In [52]:
pd.date_range("2000-01-01", periods=10, freq="1h30min")

<p>Some frequencies describe points in time that are not evenly spaced. For example, <code>"M"</code> (calendar month end) and <code>"BM"</code> (last business/weekday of month) depend on the number of days in a month and, in the latter case, whether the month ends on a weekend or not. We refer to these as <em>anchored</em> offsets.</p>

<p>Refer to <a href="#table_11.4">Table 11.4</a> for a listing of frequency codes and date offset classes available in pandas.</p>

<p><strong>Note</strong><br />
Users can define their own custom frequency classes to provide date logic not available in pandas, though the full details of that are outside the scope of this book.</p>

<h4>Week of month dates</h4>

<p>One useful frequency class is “week of month,” starting with <code>WOM</code>. This enables you to get dates like the third Friday of each month:</p>

In [53]:
monthly_dates = pd.date_range("2012-01-01", "2012-09-01", 
                              freq="WOM-3FRI")
list(monthly_dates)

<h3>Shifting (Leading and Lagging) Data</h3>

<p><em>Shifting</em> refers to moving data backward and forward through time. Both Series and DataFrame have a <code>shift</code> method for doing naive shifts forward or backward, leaving the index unmodified:</p>

In [54]:
ts = pd.Series(np.random.standard_normal(4),
               index=pd.date_range("2000-01-01", periods=4, freq="M"))
ts

In [55]:
ts.shift(2)

In [56]:
ts.shift(-2)

<p>When we shift like this, missing data is introduced either at the start or the end of the time series.</p>

<p>A common use of <code>shift</code> is computing consecutive percent changes in a time series or multiple time series as DataFrame columns. This is expressed as:</p>

<pre>ts / ts.shift(1) - 1</pre>

<p>Because naive shifts leave the index unmodified, some data is discarded. Thus if the frequency is known, it can be passed to shift to advance the timestamps instead of simply the data:</p>

In [57]:
ts.shift(2, freq="M")

<p>Other frequencies can be passed, too, giving you some flexibility in how to lead and lag the data:</p>

In [58]:
ts.shift(3, freq="D")

In [59]:
ts.shift(1, freq="90T")

<p>The <code>T</code> here stands for minutes. Note that the <code>freq</code> parameter here indicates the offset to apply to the timestamps, but it does not change the underlying frequency of the data, if any.</p>

<h3>Shifting dates with offsets</h3>

<p>The pandas date offsets can also be used with <code>datetime</code> or <code>Timestamp</code> objects:</p>

In [60]:
from pandas.tseries.offsets import Day, MonthEnd
now = datetime(2011, 11, 17)
now + 3 * Day()

<p>If you add an anchored offset like <code>MonthEnd</code>, the first increment will "roll forward" a date to the next date according to the frequency rule:</p>

In [61]:
now + MonthEnd()

In [62]:
now + MonthEnd(2)

<p>Anchored offsets can explicitly “roll” dates forward or backward by simply using their <code>rollforward</code> and <code>rollback methods</code>, respectively:</p>

In [63]:
offset = MonthEnd()
offset.rollforward(now)

In [64]:
offset.rollback(now)

<p>A creative use of date offsets is to use these methods with <code>groupby</code>:</p>

In [65]:
ts = pd.Series(np.random.standard_normal(20),
               index=pd.date_range("2000-01-15", periods=20,
                                   freq="4D"))
ts

In [66]:
ts.groupby(MonthEnd().rollforward).mean()

<p>Of course, an easier and faster way to do this is with <code>resample</code> (we'll discuss this in much more depth in <a href="#11.6_resampling">Resampling and Frequency Conversion</a>):</p>

In [67]:
ts.resample("M").mean()

<span id="11.4_time"></span>
<h2>11.4 Time Zone Handling</h2>

<p>Working with time zones can be one of the most unpleasant parts of time series manipulation. As a result, many time series users choose to work with time series in coordinated universal time or UTC, which is the geography-independent international standard. Time zones are expressed as offsets from UTC; for example, New York is four hours behind UTC during daylight saving time (DST) and five hours behind the rest of the year.</p>

<p>In Python, time zone information comes from the third-party <code>pytz</code> library (installable with pip or conda), which exposes the <em>Olson database</em>, a compilation of world time zone information. This is especially important for historical data because the DST transition dates (and even UTC offsets) have been changed numerous times depending on the regional laws. In the United States, the DST transition times have been changed many times since 1900!</p>

<p>For detailed information about the <code>pytz</code> library, you’ll need to look at that library’s documentation. As far as this book is concerned, pandas wraps <code>pytz</code>’s functionality so you can ignore its API outside of the time zone names. Since pandas has a hard dependency on <code>pytz</code>, it isn't necessary to install it separately. Time zone names can be found interactively in the docs:</p>

In [68]:
import pytz
pytz.common_timezones[-5:]

<p>To get a time zone object from <code>pytz</code>, use <code>pytz.timezone</code>:</p>

In [69]:
tz = pytz.timezone("America/New_York")
tz

<p>Methods in pandas will accept either time zone names or these objects.</p>

<h3>Time Zone Localization and Conversion</h3>

<p>By default, time series in pandas are <em>time zone naive</em>. For example, consider the following time series:</p>

In [70]:
dates = pd.date_range("2012-03-09 09:30", periods=6)
ts = pd.Series(np.random.standard_normal(len(dates)), index=dates)
ts

<p>The index's <code>tz</code> field is <code>None</code>:</p>

In [71]:
print(ts.index.tz)

<p>Date ranges can be generated with a time zone set:</p>

In [72]:
pd.date_range("2012-03-09 09:30", periods=10, tz="UTC")

<p>Conversion from naive to <em>localized</em> (reinterpreted as having been observed in a particular time zone) is handled by the <code>tz_localize</code> method:</p>

In [73]:
ts

In [74]:
ts_utc = ts.tz_localize("UTC")
ts_utc

In [75]:
ts_utc.index

<p>Once a time series has been localized to a particular time zone, it can be converted to another time zone with <code>tz_convert</code>:</p>

In [76]:
ts_utc.tz_convert("America/New_York")

<p>In the case of the preceding time series, which straddles a DST transition in the <code>America/New_York</code> time zone, we could localize to US Eastern time and convert to, say, UTC or Berlin time:</p>

In [77]:
ts_eastern = ts.tz_localize("America/New_York")
ts_eastern.tz_convert("UTC")

In [78]:
ts_eastern.tz_convert("Europe/Berlin")

<p><code>tz_localize</code> and <code>tz_convert</code> are also instance methods on <code>DatetimeIndex</code>:</p>

In [79]:
ts.index.tz_localize("Asia/Shanghai")

<p><strong>Danger</strong><br />
Localizing naive timestamps also checks for ambiguous or non-existent times around daylight saving time transitions.</p>

<h3>Operations with Time Zone-Aware Timestamp Objects</h3>

<p>Similar to time series and date ranges, individual <code>Timestamp</code> objects similarly can be localized from naive to time zone-aware and converted from one time zone to another:</p>

In [80]:
stamp = pd.Timestamp("2011-03-12 04:00")

stamp_utc = stamp.tz_localize("utc")

stamp_utc.tz_convert("America/New_York")

<p>You can also pass a time zone when creating the <code>Timestamp</code>:</p>

In [81]:
stamp_moscow = pd.Timestamp("2011-03-12 04:00", 
                            tz="Europe/Moscow")
stamp_moscow

<p>Time zone-aware <code>Timestamp</code> objects internally store a UTC timestamp value as nanoseconds since the Unix epoch (January 1, 1970), so changing the time zone does not alter the internal UTC value:</p>

In [82]:
stamp_utc.value

In [83]:
stamp_utc.tz_convert("America/New_York").value

<p>When performing time arithmetic using pandas’s <code>DateOffset</code> objects, pandas respects daylight saving time transitions where possible. Here we construct timestamps that occur right before DST transitions (forward and backward). First, 30 minutes before transitioning to DST:</p>

In [84]:
stamp = pd.Timestamp("2012-03-11 01:30", tz="US/Eastern")
stamp

In [85]:
stamp + Hour()

<p>Then, 90 minutes before transitioning out of DST:</p>

In [86]:
stamp = pd.Timestamp("2012-11-04 00:30", tz="US/Eastern")
stamp

In [87]:
stamp + 2 * Hour()

<h3>Operations Between Different Time Zones</h3>

<p>If two time series with different time zones are combined, the result will be UTC. Since the timestamps are stored under the hood in UTC, this is a straightforward operation and requires no conversion:</p>

In [88]:
dates = pd.date_range("2012-03-07 09:30", periods=10, 
                      freq="B")
ts = pd.Series(np.random.standard_normal(len(dates)), index=dates)
ts

In [89]:
ts1 = ts[:7].tz_localize("Europe/London")
ts2 = ts1[2:].tz_convert("Europe/Moscow")
result = ts1 + ts2
result.index

<p>Operations between time zone-naive and time zone-aware data are not supported and will raise an exception.</p>

<h2>11.5 Periods and Period Arithmetic</h2>

<p><em>Periods</em> represent time spans, like days, months, quarters, or years. The <code>pandas.Period</code> class represents this data type, requiring a string or integer and a supported frequency from <a href="#table_11.4">Table 11.4</a>:</p>

In [90]:
p = pd.Period("2011", freq="A-DEC") #Year end
p

<p>In this case, the <code>Period</code> object represents the full time span from January 1, 2011, to December 31, 2011, inclusive. Conveniently, adding and subtracting integers from periods has the effect of shifting their frequency:</p>

In [91]:
p + 5

In [92]:
p - 2

<p>If two periods have the same frequency, their difference is the number of units between them as a date offset:</p>

In [93]:
pd.Period("2014", freq="A-DEC") - p

<p>Regular ranges of periods can be constructed with the <code>period_range<code> function:</p>

In [94]:
periods = pd.period_range("2000-01-01", "2000-06-30", freq="M")
periods

<p>The <code>PeriodIndex</code> class stores a sequence of periods and can serve as an axis index in any pandas data structure:</p>

In [95]:
pd.Series(np.random.standard_normal(6), index=periods)

<p>If you have an array of strings, you can also use the <code>PeriodIndex</code> class, where all of its values are periods:</p>

In [96]:
values = ["2001Q3", "2002Q2", "2003Q1"]
index = pd.PeriodIndex(values, freq="Q-DEC")
index

<h3>Period Frequency Conversion</h3>

<p>Periods and <code>PeriodIndex</code> objects can be converted to another frequency with their <code>asfreq</code> method. As an example, suppose we had an annual period and wanted to convert it into a monthly period either at the start or end of the year. This can be done like so:</p>

In [97]:
p = pd.Period("2011", freq="A-DEC")
p

In [98]:
p.asfreq("M", how="start")

In [99]:
p.asfreq("M", how="end")

In [100]:
p.asfreq("M")

<p>You can think of <code>Period("2011", "A-DEC")</code> as being a sort of cursor pointing to a span of time, subdivided by monthly periods. See <a href="#figure_11.1">Figure 11.1</a> for an illustration of this. For a <em>fiscal year</em> ending on a month other than December, the corresponding monthly subperiods are different:</p>

In [101]:
p = pd.Period("2011", freq="A-JUN")
p

In [102]:
p.asfreq("M", how="start")

In [103]:
p.asfreq("M", how="end")

<figure id="figure_11.1">
    <img src="images/pda3_1101.png">
    <figcaption>Figure 11.1: Period frequency conversion illustration</figcaption>
</figure>
<p>When you are converting from high to low frequency, pandas determines the subperiod, depending on where the superperiod “belongs.” For example, in <code>A-JUN</code> frequency, the month <code>Aug-2011</code> is actually part of the <code>2012</code> period:</p>

In [104]:
p = pd.Period("Aug-2011", "M")
p.asfreq("A-JUN")

<p>Whole <code>PeriodIndex</code> objects or time series can be similarly converted with the same semantics:</p>

In [105]:
periods = pd.period_range("2006", "2009", freq="A-DEC")
ts = pd.Series(np.random.standard_normal(len(periods)), index=periods)

ts

In [106]:
ts.asfreq("M", how="start")

<p>Here, the annual periods are replaced with monthly periods corresponding to the first month falling within each annual period. If we instead wanted the last business day of each year, we can use the <code>"B"</code> frequency and indicate that we want the end of the period:</p>

In [107]:
ts.asfreq("B", how="end")

<h3>Quarterly Period Frequencies</h3>

<p>Quarterly data is standard in accounting, finance, and other fields. Much quarterly data is reported relative to a <em>fiscal year end</em>, typically the last calendar or business day of one of the 12 months of the year. Thus, the period <code>2012Q4</code> has a different meaning depending on fiscal year end. pandas supports all 12 possible quarterly frequencies as <code>Q-JAN</code> through <code>Q-DEC</code>:</p>

In [108]:
p = pd.Period("2012Q4", freq="Q-JAN")
p

<p>In the case of a fiscal year ending in January, <code>2012Q4</code> runs from November 2011 through January 2012, which you can check by converting to daily frequency:</p>

In [109]:
p.asfreq("D", how="start")

In [110]:
p.asfreq("D", how="end")

<p>See <a href="#figure_11.2">Figure 11.2</a> for an illustration.</p>
<figure id="figure_11.2">
    <img src="images/pda3_1102.png">
    <figcaption>Figure 11.2: Different quarterly frequency conventions</figcaption>
</figure>
<p>Thus, it’s possible to do convenient period arithmetic; for example, to get the timestamp at 4 P.M. on the second-to-last business day of the quarter, you could do:</p>

In [111]:
p4pm = (p.asfreq("B", how="end") - 1).asfreq("T", how="start") + 16 * 60
p4pm

In [112]:
p4pm.to_timestamp()

<p>The <code>to_timestamp</code> method returns the <code>Timestamp</code> at the start of the period by default.</p>

<p>You can generate quarterly ranges using <code>pandas.period_range</code>. The arithmetic is identical, too:</p>

In [113]:
periods = pd.period_range("2011Q3", "2012Q4", freq="Q-JAN")
ts = pd.Series(np.arange(len(periods)), index=periods)
ts

In [114]:
new_periods = (periods.asfreq("B", "end") - 1).asfreq("H", "start") + 16
ts.index = new_periods.to_timestamp()
ts

<h3>Converting Timestamps to Periods (and Back)</h3>

<p>Series and DataFrame objects indexed by timestamps can be converted to periods with the <code>to_period</code> method:</p>

In [115]:
dates = pd.date_range("2000-01-01", periods=3, freq="M")
ts = pd.Series(np.random.standard_normal(3), index=dates)
ts

In [116]:
pts = ts.to_period()
pts

<p>Since periods refer to nonoverlapping time spans, a timestamp can only belong to a single period for a given frequency. While the frequency of the new <code>PeriodIndex</code> is inferred from the timestamps by default, you can specify any supported frequency (most of those listed in <a href="#table_11.4">Table 11.4</a> are supported). There is also no problem with having duplicate periods in the result:</p>

In [117]:
dates = pd.date_range("2000-01-29", periods=6)
ts2 = pd.Series(np.random.standard_normal(6), index=dates)
ts2

In [118]:
ts2.to_period("M")

<p>To convert back to timestamps, use the <code>to_timestamp</code> method, which returns a <code>DatetimeIndex</code>:</p>

In [119]:
pts = ts2.to_period()
pts

In [120]:
pts.to_timestamp(how="end")

<h3>Creating a PeriodIndex from Arrays</h3>

<p>Fixed frequency datasets are sometimes stored with time span information spread across multiple columns. For example, in this macroeconomic dataset, the year and quarter are in different columns:</p>

In [121]:
data = pd.read_csv("examples/macrodata.csv")
data.head(5)

In [122]:
data["year"]

In [123]:
data["quarter"]

<p>By passing these arrays to <code>PeriodIndex</code> with a frequency, you can combine them to form an index for the DataFrame:</p>

In [124]:
index = pd.PeriodIndex(year=data["year"], quarter=data["quarter"],
                       freq="Q-DEC")
index

In [125]:
data.index = index
data["infl"]

<span id="11.6_resampling"></span>
<h2>11.6 Resampling and Frequency Conversion</h2>

<p><em>Resampling</em> refers to the process of converting a time series from one frequency to another. Aggregating higher frequency data to lower frequency is called <em>downsampling</em>, while converting lower frequency to higher frequency is called <em>upsampling</em>. Not all resampling falls into either of these categories; for example, converting <code>W-WED</code> (weekly on Wednesday) to <code>W-FRI</code> is neither upsampling nor downsampling.</p>

<p>pandas objects are equipped with a <code>resample</code> method, which is the workhorse function for all frequency conversion. <code>resample</code> has a similar API to <code>groupby</code>; you call <code>resample</code> to group the data, then call an aggregation function:</p>

In [126]:
dates = pd.date_range("2000-01-01", periods=100)
ts = pd.Series(np.random.standard_normal(len(dates)), index=dates)
ts

In [127]:
ts.resample("M").mean()

In [128]:
ts.resample("M", kind="period").mean()

<p><code>resample</code> is a flexible method that can be used to process large time series. The examples in the following sections illustrate its semantics and use. <a href="#table_11.5">Table 11.5</a> summarizes some of its options.</p>

<table id="table_11.5">
    <caption>Table 11.5: <code>resample</code> method arguments</caption>
    <tr>
        <th>Argument</th>
        <th>Description</th>
    </tr>
    <tr>
        <td><code>rule</code></td>
        <td>String, DateOffset, or timedelta indicating desired resampled frequency (for example, ’M', ’5min', or <code>Second(15)</code>)</td>
    </tr>
    <tr>
        <td><code>axis</code></td>
        <td>Axis to resample on; default <code>axis=0</code></td>
    </tr>
    <tr>
        <td><code>fill_method</code></td>
        <td>How to interpolate when upsampling, as in <code>"ffill"</code> or <code>"bfill"</code>; by default does no interpolation</td>
    </tr>
    <tr>
        <td><code>closed</code></td>
        <td>In downsampling, which end of each interval is closed (inclusive), <code>"right"</code> or <code>"left"</code></td>
    </tr>
    <tr>
        <td><code>label</code></td>
        <td>In downsampling, how to label the aggregated result, with the <code>"right"</code> or <code>"left"</code> bin edge (e.g., the 9:30 to 9:35 five-minute interval could be labeled <code>9:30</code> or <code>9:35</code>)</td>
    </tr>
    <tr>
        <td><code>limit</code></td>
        <td>When forward or backward filling, the maximum number of periods to fill</td>
    </tr>
    <tr>
        <td><code>kind</code></td>
        <td>Aggregate to periods (<code>"period"</code>) or timestamps (<code>"timestamp"</code>); defaults to the type of index the time series has</td>
    </tr>
    <tr>
        <td><code>convention</code></td>
        <td>When resampling periods, the convention (<code>"start"</code> or <code>"end"</code>) for converting the low-frequency period to high frequency; defaults to "start"</td>
    </tr>
    <tr>
        <td><code>origin</code></td>
        <td>The "base" timestamp from which to determine the resampling bin edges; can also be one of <code>"epoch"</code>, <code>"start"</code>, <code>"start_day"</code>, <code>"end"</code>, or <code>"end_day"</code>; see the <code>resample</code> docstring for full details</td>
    </tr>
    <tr>
        <td><code>offset</code></td>
        <td>An offset timedelta added to the origin; defaults to <code>None</code></td>
    </tr>
</table>

<h3>Downsampling</h3>

<p><em>Downsampling</em> is aggregating data to a regular, lower frequency. The data you’re aggregating doesn’t need to be fixed frequently; the desired frequency defines bin edges that are used to slice the time series into pieces to aggregate. For example, to convert to monthly, <code>"M"</code> or <code>"BM"</code>, you need to chop up the data into one-month intervals. Each interval is said to be half-open; a data point can belong only to one interval, and the union of the intervals must make up the whole time frame. There are a couple things to think about when using r<code></code>esample to downsample data:</p>

<ul>
    <li>Which side of each interval is closed</li>
    <li>How to label each aggregated bin, either with the start of the interval or the end</li>
</ul>

<p>To illustrate, let’s look at some one-minute frequency data:</p>

In [129]:
dates = pd.date_range("2000-01-01", periods=12, freq="T")
ts = pd.Series(np.arange(len(dates)), index=dates)
ts

<p>Suppose you wanted to aggregate this data into five-minute chunks or <em>bars</em> by taking the sum of each group:</p>

In [130]:
ts.resample("5min").sum()

<p>The frequency you pass defines bin edges in five-minute increments. For this frequency, by default the <code>left</code> bin edge is inclusive, so the <code>00:00</code> value is included in the <code>00:00</code> to <code>00:05</code> interval, and the <code>00:05</code> value is excluded from that interval.</p>

In [131]:
ts.resample("5min", closed="right").sum()

<p>The resulting time series is labeled by the timestamps from the left side of each bin. By passing <code>label="right"</code> you can label them with the right bin edge:</p>

In [132]:
ts.resample("5min", closed="right", label="right").sum()

<p>See <a href="#figure_11.3">Figure 11.3</a> for an illustration of minute frequency data being resampled to five-minute frequency:</p>
<figure id="figure_11.3">
    <img src="images/pda3_1103.png">
    <figcaption>Figure 11.3: Five-minute resampling illustration of closed, label conventions</figcaption>
</figure>
<p>Lastly, you might want to shift the result index by some amount, say subtracting one second from the right edge to make it more clear which interval the timestamp refers to. To do this, add an offset to the resulting index:</p>

In [133]:
from pandas.tseries.frequencies import to_offset
result = ts.resample("5min", closed="right", label="right").sum()
result.index = result.index + to_offset("-1s")
result

<h3>Open-high-low-close (OHLC) resampling</h3>

<p>In finance, a popular way to aggregate a time series is to compute four values for each bucket: the first (open), last (close), maximum (high), and minimal (low) values. By using the <code>ohlc</code> aggregate function, you will obtain a DataFrame having columns containing these four aggregates, which are efficiently computed in a single function call:</p>

In [134]:
ts = pd.Series(np.random.permutation(np.arange(len(dates))), index=dates)
ts.resample("5min").ohlc()

<h3>Upsampling and Interpolation</h3>

<p>Upsampling is converting from a lower frequency to a higher frequency, where no aggregation is needed. Let’s consider a DataFrame with some weekly data:</p>

In [135]:
frame = pd.DataFrame(np.random.standard_normal((2, 4)),
                     index=pd.date_range("2000-01-01", periods=2,
                                         freq="W-WED"),
                     columns=["Colorado", "Texas", "New York", "Ohio"])
frame

<p>When you are using an aggregation function with this data, there is only one value per group, and missing values result in the gaps. We use the <code>asfreq</code> method to convert to the higher frequency without any aggregation:</p>

In [136]:
df_daily = frame.resample("D").asfreq()
df_daily

<p>Suppose you wanted to fill forward each weekly value on the non-Wednesdays. The same filling or interpolation methods available in the <code>fillna</code> and <code>reindex</code> methods are available for resampling:</p>

In [137]:
frame.resample("D").ffill()

<p>You can similarly choose to only fill a certain number of periods forward to limit how far to continue using an observed value:</p>

In [138]:
frame.resample("D").ffill(limit=2)

<p>Notably, the new date index need not coincide with the old one at all:</p>

In [139]:
frame.resample("W-THU").ffill()

<h3>Resampling with Periods</h3>

<p>Resampling data indexed by periods is similar to timestamps:</p>

In [140]:
frame = pd.DataFrame(np.random.standard_normal((24, 4)),
                     index=pd.period_range("1-2000", "12-2001",
                                           freq="M"),
                     columns=["Colorado", "Texas", "New York", "Ohio"])
frame.head()

In [141]:
annual_frame = frame.resample("A-DEC").mean()
annual_frame

In [142]:
frame.groupby(frame.index.year).mean()

<p>Upsampling is more nuanced, as before resampling you must make a decision about which end of the time span in the new frequency to place the values. The <code>convention</code> argument defaults to <code>"start"</code> but can also be <code>"end"</code>:</p>

In [143]:
# Q-DEC: Quarterly, year ending in December
annual_frame.resample("Q-DEC").ffill()

In [144]:
annual_frame.resample("Q-DEC", convention="end").asfreq()

<p>Since periods refer to time spans, the rules about upsampling and downsampling are more rigid:</p>
<ul>
    <li>In downsampling, the target frequency must be a subperiod of the source frequency.</li>
    <li>In upsampling, the target frequency must be a superperiod of the source frequency.</li>
</ul>
<p>If these rules are not satisfied, an exception will be raised. This mainly affects the quarterly, annual, and weekly frequencies; for example, the time spans defined by <code>Q-MAR</code> only line up with <code>A-MAR</code>, <code>A-JUN</code>, <code>A-SEP</code>, and <code>A-DEC</code>:</p>

In [145]:
annual_frame.resample("Q-MAR").ffill()

<h3>Grouped Time Resampling</h3>

<p>For time series data, the <code>resample</code> method is semantically a group operation based on a time intervalization. Here's a small example table:</p>

In [146]:
N = 15
times = pd.date_range("2017-05-20 00:00", freq="1min", periods=N)
df = pd.DataFrame({"time": times,
                   "value": np.arange(N)})
df

<p>Here, we can index by <code>"time"</code> and then resample:</p>

In [147]:
df.set_index("time").resample("5min").count()

<p>Suppose that a DataFrame contains multiple time series, marked by an additional group key column:</p>

In [148]:
df2 = pd.DataFrame({"time": times.repeat(3),
                    "key": np.tile(["a", "b", "c"], N),
                    "value": np.arange(N * 3.)})
df2.head(7)

<p>To do the same resampling for each value of <code>"key"</code>, we introduce the <code>pandas.Grouper</code> object:</p>

In [149]:
time_key = pd.Grouper(freq="5min")

<p>We can then set the time index, group by <code>"key"</code> and <code>time_key</code>, and aggregate:</p>

In [150]:
resampled = (df2.set_index("time")
             .groupby(["key", time_key])
             .sum())
resampled

In [151]:
resampled.reset_index()

<p>One constraint with using <code>pandas.Grouper</code> is that the time must be the index of the Series or DataFrame.</p>

<h2>11.7 Moving Window Functions</h2>

An important class of array transformations used for time series operations are statistics and other functions evaluated over a sliding window or with exponentially decaying weights. This can be useful for smoothing noisy or gappy data. I call these <em>moving window functions</em>, even though they include functions without a fixed-length window like exponentially weighted moving average. Like other statistical functions, these also automatically exclude missing data.</p>

<p>Before digging in, we can load up some time series data and resample it to business day frequency:</p>

In [152]:
close_px_all = pd.read_csv("examples/stock_px.csv",
                           parse_dates=True, index_col=0)
close_px = close_px_all[["AAPL", "MSFT", "XOM"]]
close_px = close_px.resample("B").ffill()

<p>I now introduce the <code>rolling</code> operator, which behaves similarly to <code>resample</code> and <code>groupby</code>. It can be called on a Series or DataFrame along with a <code>window</code> (expressed as a number of periods; see next figure for the plot created):</p>

In [153]:
close_px["AAPL"].plot()
close_px["AAPL"].rolling(250).mean().plot();

<p>The expression <code>rolling(250)</code> is similar in behavior to <code>groupby</code>, but instead of grouping, it creates an object that enables grouping over a 250-day sliding window. So here we have the 250-day moving window average of Apple's stock price.</p>

<p>By default, rolling functions require all of the values in the window to be non-NA. This behavior can be changed to account for missing data and, in particular, the fact that you will have fewer than <code>window</code> periods of data at the beginning of the time series (see next figure):</p>

In [154]:
plt.figure()
std250 = close_px["AAPL"].pct_change().rolling(
    250, min_periods=10).std()
print(std250[5:12])
std250.plot();

<p>To compute an <code>expanding window mean</code>, use the <code>expanding</code> operator instead of <code>rolling</code>. The expanding mean starts the time window from the same point as the rolling window and increases the size of the window until it encompasses the whole series. An expanding window mean on the <code>std250</code> time series looks like this:</p>

In [155]:
expanding_mean = std250.expanding().mean()

<p>Calling a moving window function on a DataFrame applies the transformation to each column (see next figure):</p>

In [156]:
plt.style.use('grayscale')
close_px.rolling(60).mean().plot(logy=True);

<p>The <code>rolling</code> function also accepts a string indicating a fixed-size time offset rolling() in moving window functions rather than a set number of periods. Using this notation can be useful for irregular time series. These are the same strings that you can pass to <code>resample</code>. For example, we could compute a 20-day rolling mean like so:</p>

In [157]:
close_px.rolling("20D").mean()

<h3>Exponentially Weighted Functions</h3>

<p>An alternative to using a fixed window size with equally weighted observations is to specify a <code>constant decay</code> factor to give more weight to more recent observations. There are a couple of ways to specify the decay factor. A popular one is using a <code>span</code>, which makes the result comparable to a simple moving window function with window size equal to the span.</p>

<p>Since an exponentially weighted statistic places more weight on more recent observations, it “adapts” faster to changes compared with the equal-weighted version.</p>

<p>pandas has the <code>ewm</code> operator (which stands for exponentially weighted moving) to go along with <code>rolling</code> and <code>expanding</code>. Here’s an example comparing a 30-day moving average of Apple’s stock price with an exponentially weighted (EW) moving average with <code>span=60</code> (see next figure):</p>

In [158]:
aapl_px = close_px["AAPL"]["2006":"2007"]

ma30 = aapl_px.rolling(30, min_periods=20).mean()
ewma30 = aapl_px.ewm(span=30).mean()

aapl_px.plot(style="k-", label="Price");
ma30.plot(style="k--", label="Simple Moving Avg");
ewma30.plot(style="k-", label="EW MA");
plt.legend();

<h3>Binary Moving Window Functions</h3>

<p>Some statistical operators, like correlation and covariance, need to operate on two time series. As an example, financial analysts are often interested in a stock’s correlation to a benchmark index like the S&amp;P 500. To have a look at this, we first compute the percent change for all of our time series of interest:</p>

In [159]:
spx_px = close_px_all["SPX"]
spx_rets = spx_px.pct_change()
returns = close_px.pct_change()

<p>After we call <code>rolling</code>, the <code>corr</code> aggregation function can then compute the rolling correlation with <code>spx_rets</code> (see next figure for the resulting plot):</p>

In [160]:
corr = returns["AAPL"].rolling(125, min_periods=100).corr(spx_rets)
corr.plot();

<p>Suppose you wanted to compute the rolling correlation of the S&amp;P 500 index with many stocks at once. You could write a loop computing this for each stock like we did for Apple above, but if each stock is a column in a single DataFrame, we can compute all of the rolling correlations in one shot by calling <code>rolling</code> on the DataFrame and passing the <code>spx_rets</code> Series.</p>

<p>See next figure for the plot of the result:</p>

In [161]:
corr = returns.rolling(125, min_periods=100).corr(spx_rets)
corr.plot();

<h3>User-Defined Moving Window Functions</h3>

<p>The <code>apply</code> method on <code>rolling</code> and related methods provides a way to apply an array function of your own creation over a moving window. The only requirement is that the function produce a single value (a reduction) from each piece of the array. For example, while we can compute sample quantiles using <code>rolling(...).quantile(q)</code>, we might be interested in the percentile rank of a particular value over the sample. The <code>scipy.stats.percentileofscore</code> function does just this (see next figure for the resulting plot):</p>

In [162]:
from scipy.stats import percentileofscore
def score_at_2percent(x):
    return percentileofscore(x, 0.02)

result = returns["AAPL"].rolling(250).apply(score_at_2percent)
result.plot();